### Imports

In [ ]:
# Imports used throughout the notebook. 
# PLEASE INSTALL THE FOLLOWING PACKAGES:
from typing import Callable, Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import scipy
import seaborn as sns
import shap
from dython import nominal
from dython.nominal import associations
from imblearn.over_sampling import RandomOverSampler
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

sns.set(style="darkgrid")
sns.set(rc={'figure.figsize':(16,8)})


### General Functions

In [ ]:
def data_preprocessing(data: pd.DataFrame, categorical_features: List[str], continuous_features: List[str], target_variable: str):
    
    df = data.copy()
    
    # subset df
    #df = df[(df["Race"]=="White") | (df["Race"]=="Black")]
    df["AgeCategory"] = df["AgeCategory"].apply(mean_of_age_category)
    
    # target
    target = df[target_variable].values
    
    df_processed = df[categorical_features + continuous_features].copy() #.copy to avoid "SettingWithCopyWarning"

    # protected variables
    sex = df["Sex"].values
    age = df["AgeCategory"].values
    race = df["Race"].values
    
    df_processed = pd.get_dummies(df_processed, prefix=None, prefix_sep='_',
                                  dummy_na=False, columns=categorical_features, drop_first=True)

    return df_processed, df, target, race

def mean_of_age_category(row):
    if "older" in row:
        return 80
    else:
        return np.mean(list(map(int, row.split("-"))))

def describe_df(df, col):
    for val in df[col].unique():
        print("Race: ", val)
        display(df[df[col]==val].describe()[['BMI','PhysicalHealth','MentalHealth', 'AgeCategory', 'SleepTime']].T.style.background_gradient(cmap='Blues'))

### Loading data

In [ ]:
train = pd.read_csv('data/heart_train.csv')
val = pd.read_csv('data/heart_val.csv')
test = pd.read_csv('data/heart_test.csv')

# Select only rows with black and white race
train = train.loc[train['Race'].isin(['White', 'Black'])]
val = val.loc[val['Race'].isin(['White', 'Black'])]
test = test.loc[test['Race'].isin(['White', 'Black'])]

categorical_features = [
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "GenHealth",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"
]

continuous_features = [
    "BMI",
    "PhysicalHealth",
    "MentalHealth"
]

target_variable = "HeartDisease"

train_processed, train_original, train_target, train_race = data_preprocessing(
    train, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
val_processed, val_original, val_target, val_race = data_preprocessing(
    val, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
test_processed, test_original, test_target, test_race = data_preprocessing(
    test, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)

y_train, y_val, y_test = train_target, val_target, test_target

### EDA

In [ ]:
# Distribution of health indicators for age groups

target = "HeartDisease"
group = "Race"
subgroup = "Sex"
indexes = "AgeCategory"
categoricals = list(set(categorical_features) - set([group, subgroup]) - set(["AgeCategory"]))
continuous = list(set(continuous_features) - set(["AgeCategory"]))

fig, axes = plt.subplots(2, 1, figsize= (15, 5), sharey=True)
colors = ["#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99", "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]


for i, g in enumerate(train_original[group].unique()):
    for c, cat in enumerate(categoricals):
        sns.kdeplot(train_original[(train_original[target]=="Yes") 
                               & (train_original[cat]=="Yes")
                               & (train_original[group]==g)][indexes], alpha=1, shade=False, color=colors[c],
               label=cat, ax=axes[i])
    axes[i].set_title(f"Group: {g}")
    axes[i].set_xlabel("AgeCategory")
    axes[i].set_ylabel("Frequency")
    axes[i].legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    axes[i].set_xlim(0, 100)
plt.tight_layout()
#plt.savefig("diseases_frequency_white_black.png", dpi=400)

In [ ]:
# 

binary_features = ["Asthma_Yes", "KidneyDisease_Yes", "Smoking_Yes", "Stroke_Yes", 
                   "SkinCancer_Yes", "AlcoholDrinking_Yes", "PhysicalActivity_Yes", "DiffWalking_Yes"]

fig, ax = plt.subplots(1, 2, figsize=(14,7), sharey=True)
x = np.arange(len(binary_features))
width = 0.35

for val in train_processed["Race_White"].unique():
    no = []
    yes = []
    
    for col in binary_features:
        n, y = train_processed[train_processed["Race_White"]==val][col].value_counts(normalize=True).values
        no.append(n)
        no=[round(i, 2) for i in no]
        yes.append(y)
        yes=[round(i, 2) for i in yes]
    
    if val == 1:
        rects1 = ax[0].barh(x - width/2, no, width, label='No')
        rects2 = ax[0].barh(x + width/2, yes, width, label='Yes')
        
        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax[0].set_ylabel('Binary Features')
        ax[0].set_xlabel('Percent')

        ax[0].set_title('Race: White')
        ax[0].set_yticks(x, binary_features)
        
        ax[0].bar_label(rects1, padding=3)
        ax[0].bar_label(rects2, padding=3)
    else:
        rects1 = ax[1].barh(x - width/2, no,width, label='No')
        rects2 = ax[1].barh(x + width/2, yes, width, label='Yes')
        
        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax[1].set_title('Race: Black')
        ax[1].set_yticks(x, binary_features)
        ax[1].set_xlabel('Percent')

        
        ax[1].bar_label(rects1, padding=3)
        ax[1].bar_label(rects2, padding=3)

#ax.legend()

fig.tight_layout()
plt.legend(loc=2, labels=['No', 'Yes'], bbox_to_anchor=(1.02, 1), borderaxespad=0.)
plt.show()

### Methods

In [ ]:
# Functions and classes used in our experiments

def equailized_odds(preds: np.ndarray, groups: np.ndarray,
                    test: np.ndarray, verbose: bool = False) -> Union[float, Dict]:
    """
    Calculates the equailized odds of a binary classification problem.
    preds: predictions of the model
    groups: group labels of the test data
    test: test data
    sum_of_differences: if True, the sum of the differences is returned, else the mean of the differences is returned
    verbose: if True, prints the results
    """

    df = pd.DataFrame(list(zip(preds, groups, test)),
                      columns=['preds', 'groups', 'test'])

    # save all results
    all_results = {}

    total_class_difference = 0
    for target in df['test'].unique():
        results = {}
        for group in df['groups'].unique():

            # get the group and amount of corrects in the group
            selection = df.loc[(df['test'] == target) &
                               (df['groups'] == group)]
            corrects = selection.loc[selection['preds'] == 'Yes']

            # if there are no corrects in the group, skip
            if len(corrects) == 0:
                results[group] = 0
                continue

            # get the odds ratio
            score = round(len(corrects) / len(selection), 3)

            # add the score to the results
            results[group] = score

            if verbose:
                print(f'Target [{target}] and group [{group}]: {score} ')

        class_differences = abs(results['White'] - results['Black'])

        if verbose:
            print(results)
            print(f'Class differences for class {group}: {class_differences}')

        # sum up differences or take average
        total_class_difference += class_differences

        all_results[target] = results

    if verbose:
        print(f'Total class difference: {total_class_difference}')

    return total_class_difference, all_results

class Objective(object):
    """Objective class used in hyperparameter optimization"""
    def __init__(
        self,
        X_train: np.ndarray,
        X_val: np.ndarray,
        y_train: np.ndarray,
        y_val: np.ndarray,
        group_val: np.ndarray,
        evaluation_func: Callable,
        run_optim_no_fairness: bool = False,
    ):
        """
        Parameters
        ----------
        X_train : np.ndarray
            Training data
        X_val : np.ndarray
            Validation data
        y_train : np.ndarray
            Training labels
        y_val : np.ndarray
            Validation labels
        group_val : np.ndarray
            Validation group
        evaluation_func : Callable
            Evaluation function
        run_optim_no_fairness : bool, optional
            Run optimization without considering fairness, by default False
        """
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val
        self.group_val = group_val
        self.evaluation_func = evaluation_func,
        self.run_optim_no_fairness = run_optim_no_fairness

    def __call__(self, trial):
        """This method is called by Optuna to compute the objective
        function."""

        # Parameter space to optimize
        params = {
            "criterion": trial.suggest_categorical("criterion", ["gini"]),
            "max_depth": trial.suggest_int("max_depth", 20, 50),
            "min_samples_split": trial.suggest_float("min_samples_split", 1e-5, 0.01),
            "min_samples_leaf": trial.suggest_float("min_samples_leaf", 1e-5, 0.01),
        }

        # parameters for fitting a model
        whitebox_model = DecisionTreeClassifier(
            **params, random_state=42).fit(self.X_train, self.y_train)

        preds: np.ndarray = whitebox_model.predict(self.X_val)

        if self.run_optim_no_fairness:
            return metrics.f1_score(self.y_val, preds, labels=[
                                    'Yes'], pos_label="Yes")
        else:
            return self.evaluation_func[0](preds, self.group_val, self.y_val, verbose=False)[
                0], metrics.f1_score(self.y_val, preds, labels=['Yes'], pos_label="Yes")

def reproject_features(
        data: pd.DataFrame, protected_cols: List[str], nonprotected_cols: List[str]) -> np.ndarray:
    """
    generate a fair representation of nonprotected columns which are independent from any columns in protected_cols

    data : pd.DataFrame
        dataframe with columns to be projected
    protected_cols : List[str]
        list of protected columns
    nonprotected_cols : List[str]
        list of non-protected columns
    """
    # make a copy of data
    df: pd.DataFrame = data.copy()
    # df is our data
    # extract data about protected columns
    nonprotect: np.ndarray = df[nonprotected_cols].values
    protect: np.ndarray = df[protected_cols].values
    # extract data about nonprotected columns
    debiased_nonprotect: np.ndarray = df[nonprotected_cols].values
    # crease an orthonormal basis
    base_protect: np.ndarray = scipy.linalg.orth(protect)

    batch_size = 1000

    # go through all protected attributes and calculate their contribution to
    # the reprojection to the hyperplane
    for j in range(debiased_nonprotect.shape[1]):
        for index in range(0, data.shape[0], batch_size):
            start, stop = index, min(index + batch_size, data.shape[0])
            debiased_nonprotect[start:stop,
                                j] -= base_protect[start:stop] @ base_protect[start:stop].T @ nonprotect[start:stop, j]
    return debiased_nonprotect


def reproject_features_w_regul(
        data: pd.DataFrame, protected_cols: List[str], nonprotected_cols: List[str], lambda_: float) -> np.ndarray:
    """
    generate a fair representation of nonprotected columns which are independent from any columns in protected_cols
    data: a data frame
    protected_cols: list of strings, the protected columns
    nonprotected_col: string, all other data columns
    lambda_: float number between 0 and 1, 0 means totally fair; 1 means same as raw data
    """

    # run the normal reproject_features function
    r: np.ndarray = reproject_features(data, protected_cols, nonprotected_cols)

    # extract data about nonprotected variables
    nonprotect: np.ndarray = data[nonprotected_cols].values
    # standardize columns

    return r + lambda_ * (nonprotect - r)


### Experiment 1 and 2

In [ ]:
# Reloading the data as we overwrite the dataframe with the new data
train = pd.read_csv('data/heart_train.csv')
val = pd.read_csv('data/heart_val.csv')
test = pd.read_csv('data/heart_test.csv')

# Select only rows with black and white race
train = train.loc[train['Race'].isin(['White', 'Black'])]
val = val.loc[val['Race'].isin(['White', 'Black'])]
test = test.loc[test['Race'].isin(['White', 'Black'])]

categorical_features = [
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "GenHealth",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"
]

continuous_features = [
    "BMI",
    "PhysicalHealth",
    "MentalHealth"
]

target_variable = "HeartDisease"

train_processed, train_original, train_target, train_race = data_preprocessing(
    train, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
val_processed, val_original, val_target, val_race = data_preprocessing(
    val, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
test_processed, test_original, test_target, test_race = data_preprocessing(
    test, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)

y_train, y_val, y_test = train_target, val_target, test_target

In [ ]:
# THIS MIGHT TAKE A LITTLE WHILE TO RUN FYI.

experiment_number = 2 # select 1 or 2

if experiment_number == 1:
    standardize = True # standardize data
    standardize_continuous = True # standardize continuous data only
    resample = False # resample data to ensure even distribution of targets
    reproject = False # reproject data
    lambda_reproject = False # use regularization in reprojection
    run_optim_no_fairness = True # If true, run optimization without fairness

else:
    standardize = True
    standardize_continuous = True
    resample = True
    reproject = False
    lambda_reproject = False
    run_optim_no_fairness = False

if standardize:
    if standardize_continuous:
        # Scale continuous variables
        mean_ = np.mean(train_processed[continuous_features], axis=0)
        std_ = np.std(train_processed[continuous_features], ddof=1, axis=0)

        train_processed = (
            train_processed[continuous_features] - mean_) / std_
        val_processed = (val_processed[continuous_features] - mean_) / std_
        test_processed = (
            test_processed[continuous_features] - mean_) / std_

    else:
        # Standardize all variables
        mean_ = np.mean(train_processed, axis=0)
        std_ = np.std(train_processed, ddof=1, axis=0)

        train_processed = (train_processed - mean_) / std_
        val_processed = (val_processed - mean_) / std_
        test_processed = (test_processed - mean_) / std_

    sampler = optuna.samplers.TPESampler()

    if run_optim_no_fairness:
        study = optuna.create_study(direction="maximize",
                                    sampler=sampler,
                                    pruner=optuna.pruners.MedianPruner(
                                        n_startup_trials=2, n_warmup_steps=5,
                                        interval_steps=3),
                                    )

    else:
        study = optuna.create_study(directions=["minimize", "maximize"],
                                    sampler=sampler,
                                    pruner=optuna.pruners.MedianPruner(
                                    n_startup_trials=2, n_warmup_steps=5,
                                    interval_steps=3),
                                    )

    # Do random oversampling to make class distribution even
    if resample:
        ros = RandomOverSampler(random_state=42)
        train_processed, y_train, = ros.fit_resample(
            train_processed, y_train)

    # Define objective
    objective = Objective(
        train_processed,
        val_processed,
        y_train,
        y_val,
        val_race,
        evaluation_func=equailized_odds,
        run_optim_no_fairness=run_optim_no_fairness)

    # Make a study to optimize the objective.
    study.optimize(
        objective,
        n_trials=100,
        n_jobs=-1,
        show_progress_bar=True)

    if run_optim_no_fairness:
        print(study.best_trial)

        # Get the best parameters
        best_params = study.best_trial.params
        model = DecisionTreeClassifier(**best_params, random_state=42)
        model.fit(train_processed, y_train)

        val_pred = model.predict(val_processed)
        val_pred_proba = model.predict_proba(val_processed)
        test_pred = model.predict(test_processed)
        test_pred_proba = model.predict_proba(test_processed)

    else:
        fig = optuna.visualization.plot_pareto_front(
            study, target_names=["EO", "F1"])
        fig.show()


### Experiment 3

In [ ]:
# Reloading the data as we overwrite the dataframe with the new data
standardize = True
standardize_continuous = False
resample = True
reproject = True
lambda_reproject = False
run_optim_no_fairness = False

if reproject or lambda_reproject:
    assert standardize == True, 'Reprojecting requires standardizing'

train = pd.read_csv('data/heart_train.csv')
val = pd.read_csv('data/heart_val.csv')
test = pd.read_csv('data/heart_test.csv')

# Select only rows with black and white race
train = train.loc[train['Race'].isin(['White', 'Black'])]
val = val.loc[val['Race'].isin(['White', 'Black'])]
test = test.loc[test['Race'].isin(['White', 'Black'])]

categorical_features = [
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "GenHealth",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"
]

continuous_features = [
    "BMI",
    "PhysicalHealth",
    "MentalHealth"
]

target_variable = "HeartDisease"

train_processed, train_original, train_target, train_race = data_preprocessing(
    train, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
val_processed, val_original, val_target, val_race = data_preprocessing(
    val, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
test_processed, test_original, test_target, test_race = data_preprocessing(
    test, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)

y_train, y_val, y_test = train_target, val_target, test_target

In [ ]:
# THIS MIGHT TAKE EVEN LONGER THAN EXPERIMENT 1 and 2 AS THE REPROJECTION TAKES A LONG TIME

# Do random oversampling to make class distribution even
if resample:
    ros = RandomOverSampler(random_state=42)
    train_processed, y_train, = ros.fit_resample(
        train_processed, y_train)

protected_cols = ['Race_White']
nonprotected_cols = [
    f for f in train_processed if f not in protected_cols]

lambda_values = np.linspace(0, 1, 25)

param_grid = {'criterion': 'gini', 
            'max_depth': 30, 
            'min_samples_split': 0.008791657968284442, 
            'min_samples_leaf': 2.5756722034438575e-05}

results_f1 = {}
results_eo = {}
results_class = {
    'TPR_White': [],
    'FPR_White': [],
    'TPR_Black': [],
    'FPR_Black': []}

for i in tqdm(lambda_values, desc='Lambda'):

    #reprojection
    train_processed_r = reproject_features_w_regul(
        train_processed,
        protected_cols=protected_cols,
        nonprotected_cols=nonprotected_cols,
        lambda_=i)

    val_processed_r = reproject_features_w_regul(
        val_processed,
        protected_cols=protected_cols,
        nonprotected_cols=nonprotected_cols,
        lambda_=i)

    test_processed_r = reproject_features_w_regul(
        test_processed,
        protected_cols=protected_cols,
        nonprotected_cols=nonprotected_cols,
        lambda_=i)

    # Fit model
    whitebox_model = DecisionTreeClassifier(
        **param_grid, random_state=42).fit(train_processed_r, y_train)

    # Evaluate model
    y_pred = whitebox_model.predict(val_processed_r)

    # Calculate EO and F1 scores
    results_f1[i] = metrics.f1_score(
        y_val, y_pred, labels=['Yes'], pos_label="Yes")
    results_eo[i] = equailized_odds(y_pred, val_race, y_val)[0]

    # Save scores
    all_res = equailized_odds(y_pred, val_race, y_val)[1]
    results_class["TPR_White"].append(all_res["Yes"]['White'])
    results_class["FPR_White"].append(all_res["No"]['White'])
    results_class["TPR_Black"].append(all_res["Yes"]['Black'])
    results_class["FPR_Black"].append(all_res["No"]['Black'])

In [ ]:
# Plotting the results
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

fig.suptitle("Fairness and performance metrics for different lambda values", fontsize='x-large')
# adjust the subplots, i.e. leave more space at the top to accomodate the additional titles
fig.subplots_adjust(top=0.78)  

# plot FPR and TPR
ax[0].plot(lambda_values, results_class["TPR_White"], label='TPR White', alpha=0.6, marker='o')
ax[0].plot(lambda_values, results_class["TPR_Black"], label='TPR Black', alpha=0.6, marker='x')
ax[0].plot(lambda_values, results_class["FPR_White"], label='FPR White', alpha=0.6, marker='v')
ax[0].plot(lambda_values, results_class["FPR_Black"], label='FPR Black', alpha=0.6, marker='*')
ax[0].set_xlabel('Lambda')
ax[0].set_ylabel('FPR/TPR')
ax[0].legend(loc = 'upper left')
ax[0].set_title('FPR and TPR for the protected groups')

# Plot the results f1
ax[1].plot(lambda_values, results_f1.values(), label='F1', alpha=0.6, marker='o')
ax[1].set_xlabel('Lambda')
ax[1].set_ylabel('F1')
ax[1].legend(loc = 'upper left')
ax[1].set_title('F1 score')

plt.tight_layout()

plt.show()

### Feature importances and SHAP values

In [ ]:
# Reload data and predict on test set
standardize = True
standardize_continuous = True
resample = True
reproject = False
lambda_reproject = False
run_optim_no_fairness = False

train = pd.read_csv('data/heart_train.csv')
val = pd.read_csv('data/heart_val.csv')
test = pd.read_csv('data/heart_test.csv')

# Select only rows with black and white race
train = train.loc[train['Race'].isin(['White', 'Black'])]
val = val.loc[val['Race'].isin(['White', 'Black'])]
test = test.loc[test['Race'].isin(['White', 'Black'])]

categorical_features = [
    "Smoking",
    "AlcoholDrinking",
    "Stroke",
    "DiffWalking",
    "Sex",
    "AgeCategory",
    "Race",
    "Diabetic",
    "PhysicalActivity",
    "GenHealth",
    "Asthma",
    "KidneyDisease",
    "SkinCancer"
]

continuous_features = [
    "BMI",
    "PhysicalHealth",
    "MentalHealth"
]

target_variable = "HeartDisease"

train_processed, train_original, train_target, train_race = data_preprocessing(
    train, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
val_processed, val_original, val_target, val_race = data_preprocessing(
    val, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)
test_processed, test_original, test_target, test_race = data_preprocessing(
    test, categorical_features=categorical_features,
    continuous_features=continuous_features,
    target_variable=target_variable)

y_train, y_val, y_test = train_target, val_target, test_target

if standardize:
    if standardize_continuous:
        # Scale continuous variables
        mean_ = np.mean(train_processed[continuous_features], axis=0)
        std_ = np.std(train_processed[continuous_features], ddof=1, axis=0)

        train_processed[continuous_features] = (
            train_processed[continuous_features] - mean_) / std_
        val_processed[continuous_features] = (val_processed[continuous_features] - mean_) / std_
        test_processed[continuous_features] = (
            test_processed[continuous_features] - mean_) / std_

    else:
        # Standardize all variables
        mean_ = np.mean(train_processed, axis=0)
        std_ = np.std(train_processed, ddof=1, axis=0)

        train_processed = (train_processed - mean_) / std_
        val_processed = (val_processed - mean_) / std_
        test_processed = (test_processed - mean_) / std_

sampler = optuna.samplers.TPESampler()

# Do random oversampling to make class distribution even
if resample:
    ros = RandomOverSampler(random_state=42)
    train_processed, y_train, = ros.fit_resample(
        train_processed, y_train)

best_params_exp2 = {'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 0.008791657968284442, 'min_samples_leaf': 2.5756722034438575e-05}

model = DecisionTreeClassifier(**best_params_exp2)
model.fit(train_processed, y_train)

val_pred = model.predict(val_processed)
val_pred_proba = model.predict_proba(val_processed)
test_pred = model.predict(test_processed)
test_pred_proba = model.predict_proba(test_processed)

In [ ]:
# Feature importance
print('Feature importances')
weights = model.feature_importances_
feats_names = model.feature_names_in_
sorted_importance = list(zip(weights.tolist(), feats_names.tolist()))
sorted_importance.sort(key = lambda x: x[0], reverse=True)
sorted_importance[:10]

feature_importances = {}

feature_names = continuous_features + categorical_features

for feature in feature_names:
    feature_importances[feature] = []
    for feature_impotrance in sorted_importance:
        if feature in feature_impotrance[1]:
            feature_importances[feature].append(feature_impotrance[0])
    
feature_importances = {k: round(np.sum(v), 5) for k, v in feature_importances.items()}
feature_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

for feat, imp in feature_importances:
    print(f'{feat} has summed importance {imp}')

feats = [feat for feat, imp in feature_importances[:5]]
vals = [imp for feat, imp in feature_importances[:5]]
y_pos = np.arange(len(feats))
fig, ax = plt.subplots()
v = ax.barh(y_pos, vals, align='center')
ax.set_yticks(y_pos, labels=feats)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Importance (Gini)')
#ax.set_title('How fast do you want to go today?')


plt.show()

In [ ]:
# SHAPLEY values
explainer = shap.TreeExplainer(model, train_processed, model_output="probability")
shap_values = explainer.shap_values(test_processed)
shap.summary_plot(shap_values[1], test_processed, feature_names=train_processed.columns, max_display=10)

### Equalized odds at post-processing

In [ ]:
# Define functions

def map_array(arr:np.ndarray):
    d = {"No":0, "Yes":1}
    
    return np.array(list(map(lambda x: d[x], arr)))

import numpy as np

def check_lengths(*arrays):
    """
    Check that all arrays have the same lenght.
    Parameters
    ----------
    *arrays : list or tuple of input objects.
        Objects that will be checked.
    """
    lengths = [len(X) for X in arrays if X is not None]
    uniques = np.unique(lengths)
    if len(uniques) > 1:
        message = "Input arrays should all be the same length."
        raise ValueError(message)


def check_binaries(*arrays):
    """
    Check that all values in the arrays are 0s or 1s.
    Parameters
    ----------
    *arrays : list or tuple of input objects.
        Objects that will be checked.
    """
    values = [set(X) for X in arrays if X is not None]
    all_valid = all(v.issubset({0, 1}) for v in values)
    if not all_valid:
        message = "Input arrays should only contain 0s and/or 1s."
        raise ValueError(message)

    
def perf_measure(y_actual:np.ndarray, y_hat:np.ndarray):
    tp = 0
    fp = 0
    tn = 0
    fn = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            tp += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            fp += 1
        if y_actual[i]==y_hat[i]==0:
            tn += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            fn += 1
            
    tp = tp / (tp + fn)
    fp = fp / (tn + fp)
    tn = tn / (tn + fp)
    fn = fn / (tp + fn)

    return tp, fp, tn, fn


def perf_measure(y_actual:np.ndarray, y_hat:np.ndarray):
    tp = 0
    fp = 0
    tn = 0
    fn = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            tp += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            fp += 1
        if y_actual[i]==y_hat[i]==0:
            tn += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            fn += 1
            
    tp = tp / (tp + fn)
    fp = fp / (tn + fp)
    tn = tn / (tn + fp)
    fn = fn / (tp + fn)

    return tp, fp, tn, fn

def classification_report(y_true, y_pred, A) -> str:
    """
    String showing the true positive, false
    positive, true negatve and false negative rate 
    for each group.
    Parameters
    ----------
    y_true : 1d array of binaries
        Ground truth (correct) target values.
    y_pred : 1d array of binaries
        Estimated targets as returned by a classifier.
    groups: 1d array
        Labels for the different groups.
    """
    check_lengths(y_true, y_pred, A)
    check_binaries(y_true, y_pred)
    groups = np.unique(A)
    header = "{:<30}{:^6}{:^6}{:^6}{:^6}".format("A", "TPR", "FPR", "TNR", "FNR")
    row_fmt = "{:<30}{:^6.2f}{:^6.2f}{:^6.2f}{:^6.2f}"
    lines = [header, "-" * len(header)]
    for group in groups:
        y_true_g = y_true[A == group]
        y_pred_g = y_pred[A == group]
        tp, fp, tn, fn = perf_measure(y_true_g, y_pred_g)
        lines.append(row_fmt.format(group, tp, fp, tn, fn))

    tp, fp, tn, fn = perf_measure(y_true, y_pred)
    lines.append(row_fmt.format("All", tp, fp, tn, fn))
    report = "\n".join(lines)
    
    return report

def evaluate(y_target, y_pred, y_race):
    print(classification_report(y_target, y_pred))
    print(classification_report(y_target, y_pred, y_race))
    print(confusion_matrix(y_target, y_pred, normalize='true', labels=[1,0]))

def equalized_odds(preds, groups, test):
    df = pd.DataFrame(list(zip(preds, groups, test)), columns=['preds', 'groups', 'test'])
    targets = df['test'].unique()
    groups = df['groups'].unique()
    labels = []
    y = []
    
    
    for target in targets:
        temp_labels = []
        temp_y = []
        for group in groups:
            selection = df.loc[(df['test'] == target) & (df['groups'] == group)]
            corrects = selection.loc[selection['preds'] == target]
            score = round(len(corrects) / len(selection), 3) #how many correct out of all for this target and group
            temp_labels.append(f'T: {target}, G: {group}')
            temp_y.append(score)
        labels.append(temp_labels)
        y.append(temp_y)
    

    
    # PLOTTING
    fig, ax = plt.subplots(figsize=(6,5), sharey=True)
    
    x = np.arange(len(groups))  # the label locations
    x = np.array([i*2 for i in x])
    width = 0.5
    
    rects1 = ax.bar(x - width/2, y[0], width, label='No') #0
    rects2 = ax.bar(x + width/2, y[1], width, label='Yes') #1
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Score')
    ax.set_title('Scores by group and target')
    ax.set_xticks(x, groups)
    ax.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
    
    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    plt.tight_layout()
    plt.show()

def find_thresholds(parameters, i, fpr_goal=0.6):
    threshold = parameters["threshold"][i]
    fpr = parameters["fpr"][i]
    
    idx = np.argmin(abs(fpr_goal - parameters["fpr"][i]))
    threshold = parameters["threshold"][i][idx]
    
    return threshold, idx


def plot_roc_curve(y_race:np.ndarray, y_target:np.ndarray, y_pred_proba:np.ndarray):
    fig, ax = plt.subplots(figsize=(6,6))
    parameters = {"fpr": [], "tpr": [], "threshold": []}

    for race in np.unique(y_race):
        y_score = y_pred_proba[y_race==race]
        fpr, tpr, threshold = roc_curve(y_target[y_race==race], y_score[:, 1], drop_intermediate=False)
        roc_auc = auc(fpr, tpr)

        ax.plot(
            fpr,
            tpr,
            lw=1,
            label=f"ROC curve {race} (area = %0.2f)" % roc_auc)
        parameters["fpr"].append(fpr)
        parameters["tpr"].append(tpr)
        parameters["threshold"].append(threshold)
    plt.xlabel('fpr', fontsize=12)
    plt.ylabel('tpr', fontsize=12, rotation=0)
    
    return fig, ax, parameters

def recalculate_predictions(val_race, val_pred_proba, white_threshold, black_threshold):
    # Re-calculate predictions using new thresholds

    y_pred_fair = list()
    for race, proba in zip(val_race, val_pred_proba[:,1]):
        if race == "White":
            y_pred_fair.append(1 if proba >= white_threshold else 0)
        elif race == "Black":
            y_pred_fair.append(1 if proba >= black_threshold else 0)
    y_pred_fair = np.array(y_pred_fair)
    
    return y_pred_fair

In [ ]:
val_target = map_array(val_target)
val_pred = map_array(val_pred)
test_target = map_array(test_target)
test_pred = map_array(test_pred)

#### Experiment 1

In [ ]:
best_params_exp1 = {'criterion': 'gini', 'max_depth': 29, 'min_samples_split': 0.0015208435967915194, 'min_samples_leaf': 0.00010984751659357859}

model = DecisionTreeClassifier(**best_params_exp1, random_state=42)
model.fit(train_processed, y_train)

val_pred = model.predict(val_processed)
val_pred_proba = model.predict_proba(val_processed)
test_pred = model.predict(test_processed)
test_pred_proba = model.predict_proba(test_processed)

In [ ]:
# Plot ROC curve
fig, ax, parameters = plot_roc_curve(val_race, val_target, val_pred_proba)

fpr_goal = 0.6
threshold_black, idx_b = find_thresholds(parameters, 0, fpr_goal) #0=black
threshold_White, idx_w = find_thresholds(parameters, 1, fpr_goal) #1=white
p1 = parameters["fpr"][0][idx_b], parameters["tpr"][0][idx_b]

ax.scatter(p1[0], p1[1], s=50, color = 'green')
ax.annotate(text = f'Thresholds:\nBlack: {round(threshold_black, 3)}, White: {round(threshold_White, 3)}', xy = (p1[0]+0.01, p1[1]-0.1), size=10, color = 'black')
plt.legend()

### Experiment 2

In [ ]:
best_params_exp2 = {'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 0.008791657968284442, 'min_samples_leaf': 2.5756722034438575e-05}

model = DecisionTreeClassifier(**best_params_exp2)
model.fit(train_processed, y_train)

val_pred = model.predict(val_processed)
val_pred_proba = model.predict_proba(val_processed)
test_pred = model.predict(test_processed)
test_pred_proba = model.predict_proba(test_processed)

In [ ]:
# Plot ROC curve
fig, ax, parameters = plot_roc_curve(val_race, val_target, val_pred_proba)

fpr_goal = 0.6
threshold_black, idx_b = find_thresholds(parameters, 0, fpr_goal) #0=black
threshold_White, idx_w = find_thresholds(parameters, 1, fpr_goal) #1=white
p1 = parameters["fpr"][0][idx_b], parameters["tpr"][0][idx_b]

ax.scatter(p1[0], p1[1], s=50, color = 'green')
ax.annotate(text = f'Thresholds:\nBlack: {round(threshold_black, 3)}, White: {round(threshold_White, 3)}', xy = (p1[0]+0.01, p1[1]-0.1), size=10, color = 'black')
plt.legend()